*This notebook is part of our text recognition project for class CS345 at Colorado State University.
Original versions were created by Zachary Shimpa, Jenelle Dobyns and Jordan Rust.
The content is availabe [on GitHub](github.com/ZorkDaNerd/CS345-Text-Recognition).*

*Code help and referance was provided from Prof. Asa Ben-Hur and CS 345: Machine Learning Foundations and Practice at Colorado State University.
Original versions of these notebooks were created by Asa Ben-Hur with updates by Ross Beveridge.
The content is availabe [on his GitHub](https://github.com/asabenhur/CS345).*

<a href="https://colab.research.google.com/github/ZorkDaNerd/CS345-Text-Recognition/blob/main/Text_Recognition_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Possible data sets


https://www.kaggle.com/datasets/konradb/text-recognition-total-text-dataset


https://github.com/topics/text-recognition

https://analyticsindiamag.com/10-open-source-datasets-for-text-classification/

https://github.com/cs-chan/Total-Text-Dataset

We might have to use tesseract and opencv for like reading text inside a pdf or something, we may need to do something different for test in images

we want to do optical character recognition (OCR)
https://pyimagesearch.com/2018/09/17/opencv-ocr-and-text-recognition-with-tesseract/

maybe we use them as a guide?

# Description of Project

This project is about recognizing text from images using LSDM.

### Coding languages and packages used in project

Ex: Anaconda, Python, ect

# Project Code

In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_digits
from io import StringIO
import requests
import zipfile as zf

    
    
    

In [25]:
import opendatasets as od
text_data_url = "https://www.kaggle.com/datasets/konradb/text-recognition-total-text-dataset/download?datasetVersionNumber=1"
txtd = requests.get(text_data_url)
#txtd = zf.ZipFile(StringIO(txtd.content))

ModuleNotFoundError: No module named 'opendatasets'

In [15]:
#text_data_url = "https://www.kaggle.com/datasets/konradb/text-recognition-total-text-dataset/download?datasetVersionNumber=1/archive.zip"
#txtd = requests.get(text_data_url)
files = zf.ZipFile("archive.zip", "r")
files.extractall()
files.close()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


FileNotFoundError: [Errno 2] No such file or directory: 'archive.zip'

# Analysis of results

# Findings